# GNN for NLP
* STC-SKB-GRAPH-Ohsumed
---

In [38]:
import tqdm
import numpy as np
import json
import spacy
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Short Text Classification (STC)

In [39]:
STC_Benchmark_path = "../benchmark"
Ohsumed_training: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_training.npy", allow_pickle=True).tolist()
Ohsumed_test: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_test.npy", allow_pickle=True).tolist()
Ohsumed_category_description: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_category_description.npy", allow_pickle=True).tolist()
Ohsumed_ohsumed_all: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_ohsumed_all.npy", allow_pickle=True).tolist()

In [40]:
Ohsumed_ohsumed_all_tagme: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_ohsumed_all_tagme.npy", allow_pickle=True).tolist()
Ohsumed_training_tagme: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_training_tagme.npy", allow_pickle=True).tolist()
Ohsumed_test_tagme: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_test_tagme.npy", allow_pickle=True).tolist()
Ohsumed_category_description_tagme: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_category_description_tagme.npy", allow_pickle=True).tolist()

In [41]:
def tagmeProcess(tagme_json):
    tagme_dict = {}
    for items in tqdm.tqdm(tagme_json):
        if not items:
            continue 
        i, entityList = items.split("\t")
        if entityList == "null" or len(entityList) == 0:
            continue
        entityList = json.loads(entityList)
        entities_spot = [d['spot'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
        entities_title = [d['title'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
        entities = (entities_spot,entities_title)
        if i not in tagme_dict.keys():
            tagme_dict[i] = []
        tagme_dict[i] = entities
    return tagme_dict

In [42]:
Ohsumed_ohsumed_all_tagme = tagmeProcess(Ohsumed_ohsumed_all_tagme)
Ohsumed_training_tagme = tagmeProcess(Ohsumed_training_tagme)
Ohsumed_test_tagme = tagmeProcess(Ohsumed_test_tagme)

100%|████████████████████████████████████| 4043/4043 [00:00<00:00, 68717.84it/s]


In [43]:
print(sum([len(t) for (_,t) in Ohsumed_ohsumed_all_tagme.values()])/len(Ohsumed_ohsumed_all))
print(sum([len(t) for (_,t) in Ohsumed_training_tagme.values()])/len(Ohsumed_training))
print(sum([len(t) for (_,t) in Ohsumed_test_tagme.values()])/len(Ohsumed_test))

4.9846380931554775
4.867441167709265
4.9121939154093495


In [44]:
print(sum([len(c.split(" ")) for c in Ohsumed_ohsumed_all.keys()])/len(Ohsumed_ohsumed_all))
print(sum([len(c.split(" ")) for c in Ohsumed_training.keys()])/len(Ohsumed_training))
print(sum([len(c.split(" ")) for c in Ohsumed_test.keys()])/len(Ohsumed_test))

12.106986660835338
11.820971105153411
12.016077170418006


In [45]:
list(Ohsumed_training.keys())[5]

'Clostridium difficile invasion and toxin circulation in fatal pediatric pseudomembranous colitis.'

In [46]:
list(Ohsumed_training_tagme.values())[5]

(['Clostridium difficile',
  'invasion',
  'toxin',
  'circulation',
  'fatal',
  'pediatric',
  'pseudomembranous colitis'],
 ['Clostridium difficile (bacteria)',
  'Invasive species',
  'Toxin',
  'Circulatory system',
  'Death',
  'Pediatrics',
  'Clostridium difficile colitis'])

In [47]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemmatization(sentence):
    tokens = word_tokenize(sentence)
    tagged_sent = pos_tag(tokens)
    wnl = WordNetLemmatizer()
    lemmas_sent = []
    for tag in tagged_sent:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmas_sent.append((wnl.lemmatize(tag[0], pos = wordnet_pos),wordnet_pos))
    return lemmas_sent

In [79]:
lemmatization(list(Ohsumed_training.keys())[4])

[('Intraocular', 'a'),
 ('pressure', 'n'),
 ('change', 'n'),
 ('and', 'n'),
 ('postural', 'a'),
 ('change', 'n'),
 ('of', 'n'),
 ('intraocular', 'a'),
 ('pressure', 'n'),
 ('in', 'n'),
 ('experimentally', 'r'),
 ('induced', 'a'),
 ('Hansen', 'n'),
 ("'s", 'n'),
 ('disease', 'n'),
 ('of', 'n'),
 ('rhesus', 'n'),
 (',', 'n'),
 ('mangabey', 'n'),
 (',', 'n'),
 ('and', 'n'),
 ('African', 'n'),
 ('green', 'a'),
 ('monkey', 'n'),
 ('.', 'n')]

In [23]:
dictskb_dict = np.load("../sememe_dataset/DictSKB_dict.npy", allow_pickle=True).tolist()
dictskb_sememes = np.load("../sememe_dataset/DictSKB_sememes.npy", allow_pickle=True).tolist()

In [21]:
skb_ad_dict = np.load("../sememe_dataset/skb_ad_dict.npy", allow_pickle=True).tolist()

In [85]:
skb_ad_dict['mangabey']

[('NN', {'monkey', 'tribe'}),
 ('n', {'arboreal', 'eyelid', 'limb', 'monkey', 'tail', 'white'})]

In [86]:
dictskb_dict['mangabey']

KeyError: 'mangabey'